### Block 1: Environment Setup

In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("Environment setup complete!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

Environment setup complete!
Pandas version: 2.2.3
NumPy version: 2.2.2


### Block 2: Define file Paths
 * Objective: Load and explore the 4 essential DE-SynPUF files.
 * Files: 3 Beneficiary Summary files (2008-2010) + 1 Inpatient Claims file.

In [95]:
# Define file paths
import os

# Raw data directory
raw_data_path = "data/raw/"

# File names
beneficiary_2008 = "DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv"
beneficiary_2009 = "DE1_0_2009_Beneficiary_Summary_File_Sample_1.csv" 
beneficiary_2010 = "DE1_0_2010_Beneficiary_Summary_File_Sample_1.csv"
inpatient_claims = "DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.csv"

# Full file paths
files = {
    'beneficiary_2008': os.path.join(raw_data_path, beneficiary_2008),
    'beneficiary_2009': os.path.join(raw_data_path, beneficiary_2009),
    'beneficiary_2010': os.path.join(raw_data_path, beneficiary_2010),
    'inpatient_claims': os.path.join(raw_data_path, inpatient_claims)
}

# Check if files exist
for name, path in files.items():
    if os.path.exists(path):
        size_mb = os.path.getsize(path) / (1024 * 1024)
        print(f"✓ {name}: {size_mb:.1f} MB")
    else:
        print(f"✗ {name}: File not found")

✓ beneficiary_2008: 13.9 MB
✓ beneficiary_2009: 13.8 MB
✓ beneficiary_2010: 13.4 MB
✓ inpatient_claims: 15.9 MB


### Block 3: Initial File Inspection Function

In [96]:
def inspect_csv_structure(file_path, file_name):
    """
    Inspect CSV file structure without loading the entire file
    """
    print(f"\n{'='*50}")
    print(f"INSPECTING: {file_name}")
    print(f"{'='*50}")
    
    # Read first few rows to understand structure
    sample_df = pd.read_csv(file_path, nrows=5)
    
    print(f"Columns ({len(sample_df.columns)}):")
    for i, col in enumerate(sample_df.columns):
        print(f"  {i+1:2d}. {col}")
    
    print(f"\nSample Data (first 3 rows):")
    print(sample_df.head(3))
    
    print(f"\nData Types:")
    print(sample_df.dtypes)
    
    return sample_df.columns.tolist()

### Block 4: Inspect All Files

In [97]:
# Inspect each file structure
file_columns = {}

for name, path in files.items():
    file_columns[name] = inspect_csv_structure(path, name)


INSPECTING: beneficiary_2008
Columns (32):
   1. DESYNPUF_ID
   2. BENE_BIRTH_DT
   3. BENE_DEATH_DT
   4. BENE_SEX_IDENT_CD
   5. BENE_RACE_CD
   6. BENE_ESRD_IND
   7. SP_STATE_CODE
   8. BENE_COUNTY_CD
   9. BENE_HI_CVRAGE_TOT_MONS
  10. BENE_SMI_CVRAGE_TOT_MONS
  11. BENE_HMO_CVRAGE_TOT_MONS
  12. PLAN_CVRG_MOS_NUM
  13. SP_ALZHDMTA
  14. SP_CHF
  15. SP_CHRNKIDN
  16. SP_CNCR
  17. SP_COPD
  18. SP_DEPRESSN
  19. SP_DIABETES
  20. SP_ISCHMCHT
  21. SP_OSTEOPRS
  22. SP_RA_OA
  23. SP_STRKETIA
  24. MEDREIMB_IP
  25. BENRES_IP
  26. PPPYMT_IP
  27. MEDREIMB_OP
  28. BENRES_OP
  29. PPPYMT_OP
  30. MEDREIMB_CAR
  31. BENRES_CAR
  32. PPPYMT_CAR

Sample Data (first 3 rows):
        DESYNPUF_ID  BENE_BIRTH_DT  BENE_DEATH_DT  BENE_SEX_IDENT_CD  \
0  00013D2EFD8E45D1       19230501            NaN                  1   
1  00016F745862898F       19430101            NaN                  1   
2  0001FDD721E223DC       19360901            NaN                  2   

   BENE_RACE_CD  BENE_ESR

###  Block 5: Data Type Optimization Function


✅ Critical Fixes Included:
Chronic Condition Fix: Converts 1 → True, 2 → False for all SP_ columns
BENE_ESRD_IND Standardization: Handles both int/object types, converts 'Y'→True, '0'→False
Diagnosis Code Consistency: Handles mixed data types in ICD9 codes
Memory Optimization: Categories for appropriate columns, int32 for smaller IDs

✅ Focused Approach:
No date conversion - keeps dates as integers for now
No missing value imputation - preserves data integrity
Robust error handling - continues processing if individual conversions fail
Detailed logging - shows what's being converted and counts

✅ Benefits:
Fixes the critical chronic condition encoding issue
Standardizes ESRD indicators across years
Reduces memory usage significantly
Maintains data integrity
Provides useful feedback on data patterns

In [109]:
def optimize_dtypes(df, file_type):
    """
    Optimize data types to reduce memory usage - focused on critical fixes only
    """
    print(f"\nOptimizing data types for {file_type}...")
    
    # Store original memory usage
    original_memory = df.memory_usage(deep=True).sum() / 1024**2
    
    # Make a copy to avoid modifying original
    df_optimized = df.copy()
    
    # Optimize based on file type
    if 'beneficiary' in file_type.lower():
        print("  Applying beneficiary-specific optimizations...")
        
        # Fix chronic condition encoding: 1 = Yes (True), 2 = No (False)
        chronic_conditions = [col for col in df_optimized.columns if col.startswith('SP_') and col not in ['SP_STATE_CODE']]
        print(f"  Converting {len(chronic_conditions)} chronic condition columns...")
        
        for col in chronic_conditions:
            if col in df_optimized.columns:
                # Convert 1 -> True, 2 -> False
                df_optimized[col] = (df_optimized[col] == 1)
                print(f"    {col}: {df_optimized[col].sum()} patients with condition")
        
        # Fix BENE_ESRD_IND inconsistency (handle both int and object types)
        if 'BENE_ESRD_IND' in df_optimized.columns:
            print("  Fixing BENE_ESRD_IND encoding...")
            # Convert Y -> True, 0 -> False (handles both 'Y', 'y', 0, '0')
            df_optimized['BENE_ESRD_IND'] = df_optimized['BENE_ESRD_IND'].astype(str).str.upper()
            df_optimized['BENE_ESRD_IND'] = (df_optimized['BENE_ESRD_IND'] == 'Y')
            esrd_count = df_optimized['BENE_ESRD_IND'].sum()
            print(f"    ESRD patients: {esrd_count}")
        
        # Convert demographic categorical variables
        categorical_demos = ['BENE_SEX_IDENT_CD', 'BENE_RACE_CD', 'SP_STATE_CODE']
        for col in categorical_demos:
            if col in df_optimized.columns:
                df_optimized[col] = df_optimized[col].astype('category')
    
    elif 'inpatient' in file_type.lower():
        print("  Applying inpatient-specific optimizations...")
        
        # Convert provider and DRG codes to categories
        categorical_cols = ['PRVDR_NUM', 'CLM_DRG_CD']
        for col in categorical_cols:
            if col in df_optimized.columns:
                df_optimized[col] = df_optimized[col].astype('category')
        
        # Handle diagnosis codes - convert to string first, then category (for consistent handling)
        diagnosis_cols = [col for col in df_optimized.columns if col.startswith('ICD9_DGNS_CD_')]
        print(f"  Processing {len(diagnosis_cols)} diagnosis code columns...")
        
        for col in diagnosis_cols:
            if col in df_optimized.columns:
                # Convert to string to handle mixed types, then to category
                df_optimized[col] = df_optimized[col].astype(str)
                # Replace 'nan' strings with actual NaN
                df_optimized[col] = df_optimized[col].replace('nan', pd.NA)
                df_optimized[col] = df_optimized[col].astype('category')
        
        # Handle procedure codes similarly
        procedure_cols = [col for col in df_optimized.columns if col.startswith('ICD9_PRCDR_CD_')]
        print(f"  Processing {len(procedure_cols)} procedure code columns...")
        
        for col in procedure_cols:
            if col in df_optimized.columns:
                df_optimized[col] = df_optimized[col].astype(str)
                df_optimized[col] = df_optimized[col].replace('nan', pd.NA)
                df_optimized[col] = df_optimized[col].astype('category')
        
        # Convert integer ID columns to more efficient types
        id_cols = ['CLM_ID', 'SEGMENT', 'AT_PHYSN_NPI']
        for col in id_cols:
            if col in df_optimized.columns and df_optimized[col].dtype == 'int64':
                # Check if values fit in smaller integer types
                max_val = df_optimized[col].max()
                if max_val < 2147483647:  # fits in int32
                    try:
                        df_optimized[col] = df_optimized[col].astype('int32')
                    except:
                        pass  # Keep as int64 if conversion fails
    
    # Calculate memory savings
    new_memory = df_optimized.memory_usage(deep=True).sum() / 1024**2
    savings = original_memory - new_memory
    
    print(f"  Memory usage: {original_memory:.1f} MB -> {new_memory:.1f} MB")
    print(f"  Memory saved: {savings:.1f} MB ({savings/original_memory*100:.1f}%)")
    
    return df_optimized

### Block 6: Load 2008 Beneficiary File

In [99]:
# Load 2008 Beneficiary Summary File
print("Loading 2008 Beneficiary Summary File...")

# Load the file
df_ben_2008_raw = pd.read_csv(files['beneficiary_2008'])

print(f"Raw data loaded: {df_ben_2008_raw.shape[0]:,} rows, {df_ben_2008_raw.shape[1]} columns")
print(f"Memory usage: {df_ben_2008_raw.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Optimize data types
df_ben_2008 = optimize_dtypes(df_ben_2008_raw.copy(), 'beneficiary_2008')

print("\n2008 Beneficiary Data Summary:")
print(df_ben_2008.info())

Loading 2008 Beneficiary Summary File...
Raw data loaded: 116,352 rows, 32 columns
Memory usage: 39.4 MB

Optimizing data types for beneficiary_2008...
  Applying beneficiary-specific optimizations...
  Converting 11 chronic condition columns...
    SP_ALZHDMTA: 22410 patients with condition
    SP_CHF: 33155 patients with condition
    SP_CHRNKIDN: 18686 patients with condition
    SP_CNCR: 7415 patients with condition
    SP_COPD: 15743 patients with condition
    SP_DEPRESSN: 24840 patients with condition
    SP_DIABETES: 44060 patients with condition
    SP_ISCHMCHT: 48942 patients with condition
    SP_OSTEOPRS: 20177 patients with condition
    SP_RA_OA: 17916 patients with condition
    SP_STRKETIA: 5223 patients with condition
  Fixing BENE_ESRD_IND encoding...
    ESRD patients: 8261
  Memory usage: 39.4 MB -> 23.1 MB
  Memory saved: 16.3 MB (41.4%)

2008 Beneficiary Data Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116352 entries, 0 to 116351
Data columns (total

### Block 7: Explore 2008 Beneficiary Data

In [100]:
# Explore 2008 Beneficiary data
print("2008 BENEFICIARY DATA EXPLORATION")
print("="*50)

# Basic statistics
print(f"Total beneficiaries: {df_ben_2008.shape[0]:,}")
print(f"Unique beneficiaries: {df_ben_2008['DESYNPUF_ID'].nunique():,}")

# Demographics
print("\nDEMOGRAPHICS:")
if 'BENE_SEX_IDENT_CD' in df_ben_2008.columns:
    print("Gender distribution:")
    print(df_ben_2008['BENE_SEX_IDENT_CD'].value_counts())

if 'BENE_RACE_CD' in df_ben_2008.columns:
    print("\nRace distribution:")
    print(df_ben_2008['BENE_RACE_CD'].value_counts())

# Chronic conditions
chronic_conditions = [col for col in df_ben_2008.columns if col.startswith('SP_') and col not in ['SP_STATE_CODE']]
if chronic_conditions:
    print(f"\nCHRONIC CONDITIONS (found {len(chronic_conditions)} conditions):")
    for condition in chronic_conditions[:5]:  # Show first 5
        if condition in df_ben_2008.columns:
            count = (df_ben_2008[condition] == 1).sum()
            pct = count / len(df_ben_2008) * 100
            print(f"  {condition}: {count:,} ({pct:.1f}%)")

# Missing values
print("\nMISSING VALUES:")
missing_summary = df_ben_2008.isnull().sum()
missing_pct = (missing_summary / len(df_ben_2008)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing_summary,
    'Missing_Percentage': missing_pct
}).sort_values('Missing_Count', ascending=False)

print("Top 10 columns with missing values:")
print(missing_df.head(10))

2008 BENEFICIARY DATA EXPLORATION
Total beneficiaries: 116,352
Unique beneficiaries: 116,352

DEMOGRAPHICS:
Gender distribution:
BENE_SEX_IDENT_CD
2    64347
1    52005
Name: count, dtype: int64

Race distribution:
BENE_RACE_CD
1    96349
2    12343
3     4931
5     2729
Name: count, dtype: int64

CHRONIC CONDITIONS (found 11 conditions):
  SP_ALZHDMTA: 22,410 (19.3%)
  SP_CHF: 33,155 (28.5%)
  SP_CHRNKIDN: 18,686 (16.1%)
  SP_CNCR: 7,415 (6.4%)
  SP_COPD: 15,743 (13.5%)

MISSING VALUES:
Top 10 columns with missing values:
                          Missing_Count  Missing_Percentage
BENE_DEATH_DT                    114538           98.440938
DESYNPUF_ID                           0            0.000000
BENE_BIRTH_DT                         0            0.000000
BENE_SEX_IDENT_CD                     0            0.000000
BENE_RACE_CD                          0            0.000000
BENE_ESRD_IND                         0            0.000000
SP_STATE_CODE                         0            

### Block 8: Load Remaining Beneficiary Files

In [101]:
# Load 2009 and 2010 Beneficiary files
print("Loading 2009 and 2010 Beneficiary Summary Files...")

# Load 2009
df_ben_2009_raw = pd.read_csv(files['beneficiary_2009'])
df_ben_2009 = optimize_dtypes(df_ben_2009_raw.copy(), 'beneficiary_2009')

# Load 2010  
df_ben_2010_raw = pd.read_csv(files['beneficiary_2010'])
df_ben_2010 = optimize_dtypes(df_ben_2010_raw.copy(), 'beneficiary_2010')

print(f"\n2009 Beneficiary: {df_ben_2009.shape[0]:,} rows")
print(f"2010 Beneficiary: {df_ben_2010.shape[0]:,} rows")

# Compare beneficiary counts across years
print("\nBENEFICIARY COUNT COMPARISON:")
print(f"2008: {df_ben_2008.shape[0]:,} beneficiaries")
print(f"2009: {df_ben_2009.shape[0]:,} beneficiaries") 
print(f"2010: {df_ben_2010.shape[0]:,} beneficiaries")

# Check for beneficiaries present in all years
ben_2008_ids = set(df_ben_2008['DESYNPUF_ID'])
ben_2009_ids = set(df_ben_2009['DESYNPUF_ID'])
ben_2010_ids = set(df_ben_2010['DESYNPUF_ID'])

all_years = ben_2008_ids & ben_2009_ids & ben_2010_ids
print(f"\nBeneficiaries present in all 3 years: {len(all_years):,}")

Loading 2009 and 2010 Beneficiary Summary Files...

Optimizing data types for beneficiary_2009...
  Applying beneficiary-specific optimizations...
  Converting 11 chronic condition columns...
    SP_ALZHDMTA: 26404 patients with condition
    SP_CHF: 39438 patients with condition
    SP_CHRNKIDN: 23718 patients with condition
    SP_CNCR: 9336 patients with condition
    SP_COPD: 17925 patients with condition
    SP_DEPRESSN: 28085 patients with condition
    SP_DIABETES: 47709 patients with condition
    SP_ISCHMCHT: 54601 patients with condition
    SP_OSTEOPRS: 21908 patients with condition
    SP_RA_OA: 19994 patients with condition
    SP_STRKETIA: 6030 patients with condition
  Fixing BENE_ESRD_IND encoding...
    ESRD patients: 10856
  Memory usage: 38.8 MB -> 22.7 MB
  Memory saved: 16.1 MB (41.4%)

Optimizing data types for beneficiary_2010...
  Applying beneficiary-specific optimizations...
  Converting 11 chronic condition columns...
    SP_ALZHDMTA: 18689 patients with cond

### Block 9: Load Inpatient Claims

In [102]:
# Load Inpatient Claims file
print("Loading Inpatient Claims File...")

# Load the file
df_inpatient_raw = pd.read_csv(files['inpatient_claims'])

print(f"Raw inpatient data loaded: {df_inpatient_raw.shape[0]:,} rows, {df_inpatient_raw.shape[1]} columns")
print(f"Memory usage: {df_inpatient_raw.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Optimize data types
df_inpatient = optimize_dtypes(df_inpatient_raw.copy(), 'inpatient_claims')

print("\nInpatient Claims Data Summary:")
print(df_inpatient.info())

Loading Inpatient Claims File...
Raw inpatient data loaded: 66,773 rows, 81 columns
Memory usage: 88.7 MB

Optimizing data types for inpatient_claims...
  Applying inpatient-specific optimizations...
  Processing 10 diagnosis code columns...
  Processing 6 procedure code columns...
  Memory usage: 88.7 MB -> 43.5 MB
  Memory saved: 45.1 MB (50.9%)

Inpatient Claims Data Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66773 entries, 0 to 66772
Data columns (total 81 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   DESYNPUF_ID                     66773 non-null  object  
 1   CLM_ID                          66773 non-null  int64   
 2   SEGMENT                         66773 non-null  int32   
 3   CLM_FROM_DT                     66705 non-null  float64 
 4   CLM_THRU_DT                     66705 non-null  float64 
 5   PRVDR_NUM                       66773 non-null  category
 6   CLM_PMT_

### Block 10: Explore Inpatient Claims Data

In [105]:
# Explore Inpatient Claims data
print("INPATIENT CLAIMS DATA EXPLORATION")
print("="*50)

# Basic statistics
print(f"Total inpatient claims: {df_inpatient.shape[0]:,}")
print(f"Unique beneficiaries with inpatient claims: {df_inpatient['DESYNPUF_ID'].nunique():,}")
print(f"Claims per beneficiary (avg): {df_inpatient.shape[0] / df_inpatient['DESYNPUF_ID'].nunique():.1f}")

# Date range analysis (working with integer dates)
if 'CLM_FROM_DT' in df_inpatient.columns and 'CLM_THRU_DT' in df_inpatient.columns:
    print(f"\nDATE RANGE (YYYYMMDD format):")
    print(f"Earliest admission: {df_inpatient['CLM_FROM_DT'].min()}")
    print(f"Latest discharge: {df_inpatient['CLM_THRU_DT'].max()}")

# Year distribution (extract from integer dates)
if 'CLM_THRU_DT' in df_inpatient.columns:
    # Extract year from YYYYMMDD integer format
    df_inpatient['discharge_year'] = df_inpatient['CLM_THRU_DT'] // 10000
    print(f"\nCLAIMS BY YEAR:")
    print(df_inpatient['discharge_year'].value_counts().sort_index())

# Length of stay analysis
if 'CLM_UTLZTN_DAY_CNT' in df_inpatient.columns:
    print(f"\nLENGTH OF STAY STATISTICS:")
    los_stats = df_inpatient['CLM_UTLZTN_DAY_CNT'].describe()
    print(los_stats)

# Check for readmissions (basic check)
if 'NCH_BENE_DSCHRG_DT' in df_inpatient.columns:
    # Sort by beneficiary and discharge date (integer sorting works chronologically for YYYYMMDD)
    df_sorted = df_inpatient.sort_values(['DESYNPUF_ID', 'NCH_BENE_DSCHRG_DT'])
    
    # Count beneficiaries with multiple admissions
    multiple_admissions = df_sorted['DESYNPUF_ID'].value_counts()
    readmission_candidates = (multiple_admissions > 1).sum()
    
    print(f"\nREADMISSION INDICATORS:")
    print(f"Beneficiaries with multiple admissions: {readmission_candidates:,}")
    print(f"Potential readmission rate: {readmission_candidates / df_inpatient['DESYNPUF_ID'].nunique() * 100:.1f}%")

# Display sample date formats for verification
print(f"\nSAMPLE DATE FORMATS:")
print(f"Admission dates (first 5): {df_inpatient['CLM_ADMSN_DT'].head().tolist()}")
print(f"Discharge dates (first 5): {df_inpatient['NCH_BENE_DSCHRG_DT'].head().tolist()}")

INPATIENT CLAIMS DATA EXPLORATION
Total inpatient claims: 66,773
Unique beneficiaries with inpatient claims: 37,780
Claims per beneficiary (avg): 1.8

DATE RANGE (YYYYMMDD format):
Earliest admission: 20071127.0
Latest discharge: 20101231.0

CLAIMS BY YEAR:
discharge_year
2008.0    27496
2009.0    25293
2010.0    13916
Name: count, dtype: int64

LENGTH OF STAY STATISTICS:
count    66705.000000
mean         5.582895
std          6.284463
min          0.000000
25%          2.000000
50%          4.000000
75%          7.000000
max        136.000000
Name: CLM_UTLZTN_DAY_CNT, dtype: float64

READMISSION INDICATORS:
Beneficiaries with multiple admissions: 15,168
Potential readmission rate: 40.1%

SAMPLE DATE FORMATS:
Admission dates (first 5): [20100312, 20090412, 20090831, 20090917, 20100626]
Discharge dates (first 5): [20100313, 20090418, 20090902, 20090920, 20100701]


### Code Block 11: Data Linking Validation

In [106]:
# Validate data linking between files
print("DATA LINKING VALIDATION")
print("="*50)

# Check DESYNPUF_ID consistency
inpatient_ids = set(df_inpatient['DESYNPUF_ID'])
ben_2008_ids = set(df_ben_2008['DESYNPUF_ID'])
ben_2009_ids = set(df_ben_2009['DESYNPUF_ID'])
ben_2010_ids = set(df_ben_2010['DESYNPUF_ID'])

print("LINKING ANALYSIS:")
print(f"Unique beneficiaries in inpatient claims: {len(inpatient_ids):,}")
print(f"Unique beneficiaries in 2008 summary: {len(ben_2008_ids):,}")

# Check overlap
inpatient_in_2008 = inpatient_ids & ben_2008_ids
inpatient_in_2009 = inpatient_ids & ben_2009_ids  
inpatient_in_2010 = inpatient_ids & ben_2010_ids

print(f"\nLINKING SUCCESS RATES:")
print(f"Inpatient beneficiaries found in 2008 summary: {len(inpatient_in_2008):,} ({len(inpatient_in_2008)/len(inpatient_ids)*100:.1f}%)")
print(f"Inpatient beneficiaries found in 2009 summary: {len(inpatient_in_2009):,} ({len(inpatient_in_2009)/len(inpatient_ids)*100:.1f}%)")
print(f"Inpatient beneficiaries found in 2010 summary: {len(inpatient_in_2010):,} ({len(inpatient_in_2010)/len(inpatient_ids)*100:.1f}%)")

# Check for orphaned records
orphaned_inpatient = inpatient_ids - (ben_2008_ids | ben_2009_ids | ben_2010_ids)
if orphaned_inpatient:
    print(f"\nWARNING: {len(orphaned_inpatient)} inpatient beneficiaries not found in any beneficiary summary file")
else:
    print(f"\n✓ All inpatient beneficiaries found in beneficiary summary files")

DATA LINKING VALIDATION
LINKING ANALYSIS:
Unique beneficiaries in inpatient claims: 37,780
Unique beneficiaries in 2008 summary: 116,352

LINKING SUCCESS RATES:
Inpatient beneficiaries found in 2008 summary: 37,780 (100.0%)
Inpatient beneficiaries found in 2009 summary: 37,648 (99.7%)
Inpatient beneficiaries found in 2010 summary: 37,298 (98.7%)

✓ All inpatient beneficiaries found in beneficiary summary files


### Block 12: Save Processed Files

In [107]:
# Save processed files
processed_path = "data/processed/"
os.makedirs(processed_path, exist_ok=True)

print("SAVING PROCESSED FILES")
print("="*30)

# Save beneficiary files
df_ben_2008.to_csv(f"{processed_path}beneficiary_2008_processed.csv", index=False)
df_ben_2009.to_csv(f"{processed_path}beneficiary_2009_processed.csv", index=False)
df_ben_2010.to_csv(f"{processed_path}beneficiary_2010_processed.csv", index=False)

# Save inpatient file
df_inpatient.to_csv(f"{processed_path}inpatient_claims_processed.csv", index=False)

print("✓ beneficiary_2008_processed.csv")
print("✓ beneficiary_2009_processed.csv") 
print("✓ beneficiary_2010_processed.csv")
print("✓ inpatient_claims_processed.csv")

# Also save as parquet for faster loading
df_ben_2008.to_parquet(f"{processed_path}beneficiary_2008_processed.parquet")
df_ben_2009.to_parquet(f"{processed_path}beneficiary_2009_processed.parquet")
df_ben_2010.to_parquet(f"{processed_path}beneficiary_2010_processed.parquet")
df_inpatient.to_parquet(f"{processed_path}inpatient_claims_processed.parquet")

print("✓ Parquet files saved for faster loading")

SAVING PROCESSED FILES
✓ beneficiary_2008_processed.csv
✓ beneficiary_2009_processed.csv
✓ beneficiary_2010_processed.csv
✓ inpatient_claims_processed.csv
✓ Parquet files saved for faster loading


### Code Block 13: Final Summary

In [108]:
# Final summary
print("\n" + "="*50)
print("DATA LOADING SUMMARY")
print("="*50)

print(f"FILES SUCCESSFULLY LOADED: 4/4")
print(f"├── Beneficiary 2008: {df_ben_2008.shape[0]:,} records")
print(f"├── Beneficiary 2009: {df_ben_2009.shape[0]:,} records") 
print(f"├── Beneficiary 2010: {df_ben_2010.shape[0]:,} records")
print(f"└── Inpatient Claims: {df_inpatient.shape[0]:,} records")

print(f"\nKEY STATISTICS:")
print(f"├── Total beneficiaries (2008): {df_ben_2008['DESYNPUF_ID'].nunique():,}")
print(f"├── Beneficiaries with inpatient claims: {df_inpatient['DESYNPUF_ID'].nunique():,}")
print(f"├── Total inpatient admissions: {df_inpatient.shape[0]:,}")
print(f"└── Average claims per patient: {df_inpatient.shape[0] / df_inpatient['DESYNPUF_ID'].nunique():.1f}")

print(f"\nNEXT STEPS:")
print(f"├── Ready for Notebook 2: Data Combination & Preprocessing")
print(f"├── Combine beneficiary files across years")
print(f"├── Create 30-day readmission target variable")
print(f"└── Begin feature engineering")

print(f"\n✓ Phase A Complete - Data Loading Successful!")


DATA LOADING SUMMARY
FILES SUCCESSFULLY LOADED: 4/4
├── Beneficiary 2008: 116,352 records
├── Beneficiary 2009: 114,538 records
├── Beneficiary 2010: 112,754 records
└── Inpatient Claims: 66,773 records

KEY STATISTICS:
├── Total beneficiaries (2008): 116,352
├── Beneficiaries with inpatient claims: 37,780
├── Total inpatient admissions: 66,773
└── Average claims per patient: 1.8

NEXT STEPS:
├── Ready for Notebook 2: Data Combination & Preprocessing
├── Combine beneficiary files across years
├── Create 30-day readmission target variable
└── Begin feature engineering

✓ Phase A Complete - Data Loading Successful!
